<a href="https://colab.research.google.com/github/DynamicLLM/LLM2024/blob/main/src/sample-ai-agent/Financial_AI_Agent_with_OpenAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Simplified Financial AI Agent

This notebook demonstrates how to use the OpenAI API and Yahoo Finance data to analyze financial trends, generate insights, and fetch relevant news articles.

## Key Features:
1. **Analyze Financial Data**: Use OpenAI to interpret and generate insights based on financial queries.
2. **Fetch Yahoo Finance Data**: Extract stock prices, changes, and volumes from Yahoo Finance.
3. **Stock Trend Analysis**: Combine Yahoo Finance data and OpenAI to analyze stock trends and suggest investment strategies.
4. **Fetch Yahoo News Articles**: Retrieve relevant news articles for a specific financial topic.
    

In [1]:

# Install necessary libraries
!pip install openai beautifulsoup4 requests -q


In [2]:

# Import required modules
from openai import OpenAI
import requests
from bs4 import BeautifulSoup

# Set up the OpenAI client
client = OpenAI(
    api_key="sk-****wA"
)


In [7]:

def analyze_financial_data_with_openai(prompt):
    """
    Use OpenAI to analyze financial data or answer financial questions.
    Args:
        prompt (str): The financial-related query or analysis prompt.

    Returns:
        str: Response from OpenAI's API.
    """
    try:
        completion = client.chat.completions.create(
            model="gpt-4o-mini",
            store=True,
            messages=[
                {"role": "user", "content": prompt}
            ]
        )
        return completion.choices[0].message.content
    except Exception as e:
        return f"An error occurred: {e}"



In [8]:

# Function to fetch Yahoo Finance stock data
def fetch_yahoo_finance_data(ticker):
    """
    Fetch basic stock data from Yahoo Finance for a given ticker.
    Args:
        ticker (str): Stock ticker symbol.

    Returns:
        dict: Stock data including price, change, and volume.
    """
    base_url = f"https://finance.yahoo.com/quote/{ticker}"
    try:
        response = requests.get(base_url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")
        price = soup.find("fin-streamer", {"data-field": "regularMarketPrice"})
        change = soup.find("fin-streamer", {"data-field": "regularMarketChange"})
        volume = soup.find("td", {"data-test": "TD_VOLUME-value"})
        return {
            "price": price.text if price else "N/A",
            "change": change.text if change else "N/A",
            "volume": volume.text if volume else "N/A"
        }
    except Exception as e:
        return {"error": f"An error occurred while fetching data: {e}"}


In [9]:

# Function to perform stock trend analysis
def analyze_stock_trends_with_openai(ticker):
    """
    Analyze recent trends of a stock using OpenAI.
    Args:
        ticker (str): Stock ticker symbol.

    Returns:
        str: Analysis from OpenAI.
    """
    data = fetch_yahoo_finance_data(ticker)
    if "error" in data:
        return data["error"]

    prompt = (
        f"Analyze the stock data for {ticker}. Current price: {data['price']}, "
        f"Change: {data['change']}, Volume: {data['volume']}. "
        "Provide insights and suggest an investment strategy."
    )
    return analyze_financial_data_with_openai(prompt)


In [10]:

# Example usage: Analyze financial data
financial_query = "What are the implications of rising interest rates on the stock market?"
ai_response = analyze_financial_data_with_openai(financial_query)
print("AI Analysis:", ai_response)

# Analyze stock trends for AAPL using OpenAI
aapl_analysis = analyze_stock_trends_with_openai("AAPL")
print("AAPL Analysis by OpenAI:", aapl_analysis)


AI Analysis: Rising interest rates can have several implications for the stock market, both in the short term and long term. Here are some of the key effects:

1. **Higher Borrowing Costs**: As interest rates rise, the cost of borrowing increases for both consumers and corporations. Higher borrowing costs can lead to reduced consumer spending and business investment, which may negatively impact corporate earnings and, in turn, stock prices.

2. **Discounted Cash Flows**: Stock valuations are often derived from the present value of future cash flows. Higher interest rates increase the discount rate used in these calculations, which can lead to lower present values for stocks. Growth stocks, in particular, can be more sensitive to interest rate changes since they tend to rely heavily on future earnings growth.

3. **Sector Rotation**: Rising interest rates can lead to a rotation out of certain sectors of the stock market. For instance, interest-sensitive sectors like utilities and real e

In [11]:

# Fetch Yahoo News Articles
def fetch_yahoo_news(topic):
    """
    Fetch news articles from Yahoo related to the given topic.
    Args:
        topic (str): The topic to search for news.

    Returns:
        list: A list of dictionaries containing title and link of news articles.
    """
    base_url = "https://news.search.yahoo.com/search"
    params = {"p": topic, "fr": "news"}
    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")
        articles = []
        for item in soup.find_all("div", class_="NewsArticle"):
            title = item.find("h4").text if item.find("h4") else "No Title"
            link = item.find("a")["href"] if item.find("a") else "No Link"
            articles.append({"title": title, "link": link})
        return articles
    except Exception as e:
        return [f"An error occurred while fetching news: {e}"]

news_topic = "Stock Market Trends"
news_articles = fetch_yahoo_news(news_topic)
print("Yahoo News Articles:")
for article in news_articles:
    print(f"- {article['title']} ({article['link']})")


Yahoo News Articles:
- Microsoft's Stargate Supercomputer Is an Inevitable Product of the AI Arms Race (https://finance.yahoo.com/news/microsofts-stargate-supercomputer-inevitable-product-151538470.html?fr=sycsrp_catchall)
- Is Boise Cascade Company (BCC) the Best Lumber Stock to Buy Right Now? (https://finance.yahoo.com/news/boise-cascade-company-bcc-best-151519319.html?fr=sycsrp_catchall)
- Q3 Earnings Review: Beverages, Alcohol, and Tobacco Stocks Led by Zevia... (https://finance.yahoo.com/news/q3-earnings-review-beverages-alcohol-090157967.html?fr=sycsrp_catchall)
- ZenaTech Inc (ZENA) Stock: A Guide to the Market Trend – News Heater (https://newsheater.com/2025/01/02/zenatech-inc-zena-stock-a-guide-to-the-market-trend/)
- I’m a Financial Advisor: I’d Invest My First $5,000 in These 6 Stocks (https://finance.yahoo.com/news/m-financial-advisor-d-invest-150041485.html?fr=sycsrp_catchall)
- 10 Stock Market Predictions for 2025 (https://finance.yahoo.com/news/10-stock-market-prediction